In [398]:
import pandas as pd
import openpyxl
import xlwings as xw
import xlsxwriter
import math
import numpy as np

paco_Testplan_path = 'C:/Users/oltigash/PycharmProjects/python_hf/CB_Migration/Datenbasis/Export_1 - Testplan/PaCo_Testplan.xlsx'
Template_RAW_path = 'C:/Users/oltigash/PycharmProjects/python_hf/CB_Migration/Zielformat/Template_PreFilled.xlsx'

df_x = pd.read_excel(paco_Testplan_path)
df_y = pd.read_excel(Template_RAW_path, sheet_name='Raw Data')


length = 0
list_len = []
final_list = []

if df_x['Case Id'][0] is not None:
    length += 1
for indx, value in enumerate(df_x['Case Id'], start = 1):
    if math.isnan(value):
        length +=1
    else:
        list_len.append(length)
        length = 1

element_value = 0
for value in range(len(list_len)):
    element_value += list_len[value]
    final_list.append(element_value)
#     print('old version of final list', final_list)
final_list = [x-1 for x in final_list]
final_list.append(final_list[-1] + 50)
print('new version of final list', final_list)



df_x['Step Index'] = df_x['Step Index'].astype(str)

# df_x.loc[df_x['Step Index'].notnull() & df_x['Step Action'].isnull(), 'Step Action'] = "##"

# df_x['merged'] = df_x['Step Index'].fillna('') + df_x['Step Action'].fillna('')


df_x['Step Expected'].fillna('##', inplace=True)

# df_x = df_x.assign(ExpectedResults = df_x['Step Index'][1:] + ' ' + df_x['Step Expected'][1:])
# df_x = df_x.assign(StepActions = df_x['Step Index'][1:] + ' ' + df_x['Step Action'][1:])
# df_x['ExpectedResults'] = df_x['ExpectedResults'].str.replace('\n', '')
# df_x['StepActions'] = df_x['StepActions'].str.replace('\n', '')



# df_x = df_x.fillna(np.nan)
df_x = df_x.rename(columns={"Case Id": "Id", "Case Title": "Title"})
df_x = df_x.replace(np.nan, '')
#df_x = df_x.apply(lambda row: str(row['Step Index']) +  str(row['Step Action']) if not np.isnan(row['Step Action']) else str(row['Step Index']) + '##' + 'nan', axis=1)


new version of final list [0, 15, 30, 59, 78, 115, 126, 156, 178, 201, 227, 253, 303]


In [399]:
df_x['Id'] = df_x['Id'].astype(str)
# df_x['StepActions'] = df_x['StepActions'].astype(str)

In [400]:
df_names = []
for i in range(len(final_list) - 1):
    start = final_list[i]
    end = final_list[i + 1]
    var_name = f"geteilt_df_{i}"
    locals()[var_name] = df_x[start:end]
    df_names.append(var_name)
    df = locals()[var_name]
    df['Id'] = df['Id'][start]
#     var_name = var_name.groupby('Id')['StepActions'].apply(' '.join).reset_index()    


C:\Users\oltigash\AppData\Local\Temp\ipykernel_12936\2284737326.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Id'] = df['Id'][start]


In [401]:
single_row_dfs = []
for df in df_names:
    var_name = f"single_row_df_{df}"
    name = eval(df)
#     locals()[var_name] = name.pivot_table(index='Id', aggfunc=lambda x: ' '.join(map(str, x))).reset_index()
    locals()[var_name] = name.pivot_table(index='Id', aggfunc=lambda x: '\n'.join(map(str, x))).reset_index()
    single_row_dfs.append(var_name)

In [402]:
concatenated_df_list = []
for name in single_row_dfs:
    df = eval(name)
    concatenated_df_list.append(df)
concatenated_df = pd.concat(concatenated_df_list)

In [403]:
test = pd.concat([df_y, concatenated_df])

In [404]:
test = test[1:]

In [405]:
test = test.assign(WorkItemType = 'Test Case')
test = test.assign(TeamProject = '##')
test = test.assign(State = 'Ready')
test = test.assign(AreaPath = 'Integration')

In [406]:
columns_of_df_y = []
for column in df_y.columns:
    columns_of_df_y.append(column)

In [407]:
test = test[columns_of_df_y]

In [408]:
test = test.fillna('##')

In [409]:
merged_df = []

for i in range(len(final_list) - 1):
    start = final_list[i]
    end = final_list[i+1]
    var_name = f"geteilt_dfM_{i}"
    locals()[var_name] = df_x[['Id', 'Step Index', 'Step Expected']][start:end]
    
    df = locals()[var_name]
    df['Id'] = df['Id'][start]
    merged_df.append(var_name)
    

In [410]:
geteilt_dfM_0

,Id,Step Index,Step Expected
0,513898.0,nan,##
1,513898.0,1.0,##
2,513898.0,1.1,##
3,513898.0,1.2,##
4,513898.0,1.3,##
5,513898.0,2.0,##
6,513898.0,2.1,"Param_PaCoAPS == ""APS"""
7,513898.0,2.2,"Param_PaCoOPS == ""installed"""
8,513898.0,3.0,##
9,513898.0,3.1,PDU Parkhilfe_04 is present on VLAN_FAS instance


In [411]:
single_row_dfsM = []
for df in merged_df:
    var_name = f"single_row_dfM_{df}"
    name = eval(df)
    name = name[1:]
#     locals()[var_name] = name.pivot_table(index='Id', aggfunc=lambda x: ' '.join(map(str, x))).reset_index()
    name = name.assign(ExpectedResults = name['Step Index'] + ' ' + name['Step Expected'])
    name = name.assign(StepActions = df_x['Step Index'] + ' ' + df_x['Step Action'])
    locals()[var_name] = name.pivot_table(index='Id', aggfunc=lambda x: '\n'.join(map(str, x))).reset_index()
    single_row_dfsM.append(var_name)

In [412]:
concatenated_df_listM = []
for name in single_row_dfsM:
    df = eval(name)
    concatenated_df_listM.append(df)
concatenated_dfM = pd.concat(concatenated_df_listM)

In [413]:
df_y['Id'] = df_y['Id'].astype(str)
concatenated_dfM['Id'] = concatenated_dfM['Id'].astype(str)

In [414]:
concatenated_df = concatenated_df[1:]
concatenated_dfM = concatenated_dfM[1:]

In [415]:
dfs = [df_y, concatenated_df,concatenated_dfM]
test_con = pd.concat([ concatenated_df,concatenated_dfM])

In [416]:
merged_df = pd.merge(concatenated_df, concatenated_dfM, on='Id')

In [417]:
test_con = pd.concat([df_y, merged_df])

In [423]:
final_df = test_con[columns_of_df_y]
final_df = final_df.fillna('##')

In [425]:
final_df = final_df.assign(WorkItemType = 'Test Case')
final_df = final_df.assign(TeamProject = '##')
final_df = final_df.assign(State = 'Ready')
final_df = final_df.assign(AreaPath = 'Integration')

In [429]:
final_df

,Id,WorkItemType,TeamProject,Title,State,Reason,AreaPath,IterationPath,CreatedDate,CreatedBy,...,AU_temp_import_order,AU_temp_testcase_ct_mapping,AU_temp_linked_requirements_ids,AU_temp_linked_requirements_extra_ids,AU_temp_linked_requirements_extra_1_ids,AU_temp_Linked_ADO_IDs,AU_temp_linked_jira_keys,AU_xml_preview_repo,Relations,AU_integration_kpm_id
0,513898,Test Case,##,TC_PaCo_Valid_Signal_Video_Display_Request_Par...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
0,513899.0,Test Case,##,TC_PaCo_Valid_Signal_Video_Display_Request_Rev...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
1,530082.0,Test Case,##,TC_PaCo_OPS_Act_RGear_PaCo_Error\n\n\n\n\n\n\n...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
2,530094.0,Test Case,##,TC_PaCo_OPS_Act_RGear_4KOPS_Drivingtube\n\n\n\...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
3,530092.0,Test Case,##,TC_PaCo_OPS_Act_RGear_PaCo_Init\n\n\n\n\n\n\n\...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
4,555302.0,Test Case,##,TC_PaCo_OPS_ErrorHandling_ACTPrevention_ParkKe...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
5,530146.0,Test Case,##,TC_PaCo_OPS_Act_Hardkey_Velocityhystereses\n\n...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
6,530148.0,Test Case,##,TC_PaCo_OPS_Act_Hardkey8KOPS_Drivingtube\n\n\n...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
7,530151.0,Test Case,##,TC_PaCo_OPS_Act_Hardkey12KOPS_Drivingtube\n\n\...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
8,542947.0,Test Case,##,TC_PaCo_OPS_Deact_HardkeyFromR_Gear\n\n\n\n\n\...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##


In [431]:
final_df = final_df[columns_of_df_y]
final_df

,Id,WorkItemType,TeamProject,Title,State,Reason,AreaPath,IterationPath,CreatedDate,CreatedBy,...,AU_temp_import_order,AU_temp_testcase_ct_mapping,AU_temp_linked_requirements_ids,AU_temp_linked_requirements_extra_ids,AU_temp_linked_requirements_extra_1_ids,AU_temp_Linked_ADO_IDs,AU_temp_linked_jira_keys,AU_xml_preview_repo,Relations,AU_integration_kpm_id
0,513898,Test Case,##,TC_PaCo_Valid_Signal_Video_Display_Request_Par...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
0,513899.0,Test Case,##,TC_PaCo_Valid_Signal_Video_Display_Request_Rev...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
1,530082.0,Test Case,##,TC_PaCo_OPS_Act_RGear_PaCo_Error\n\n\n\n\n\n\n...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
2,530094.0,Test Case,##,TC_PaCo_OPS_Act_RGear_4KOPS_Drivingtube\n\n\n\...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
3,530092.0,Test Case,##,TC_PaCo_OPS_Act_RGear_PaCo_Init\n\n\n\n\n\n\n\...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
4,555302.0,Test Case,##,TC_PaCo_OPS_ErrorHandling_ACTPrevention_ParkKe...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
5,530146.0,Test Case,##,TC_PaCo_OPS_Act_Hardkey_Velocityhystereses\n\n...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
6,530148.0,Test Case,##,TC_PaCo_OPS_Act_Hardkey8KOPS_Drivingtube\n\n\n...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
7,530151.0,Test Case,##,TC_PaCo_OPS_Act_Hardkey12KOPS_Drivingtube\n\n\...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##
8,542947.0,Test Case,##,TC_PaCo_OPS_Deact_HardkeyFromR_Gear\n\n\n\n\n\...,Ready,##,Integration,##,##,##,...,##,##,##,##,##,##,##,##,##,##


In [428]:
final_df.to_excel('final_test.xlsx')